In [1]:
import random
from decimal import *
import numpy as np
import collections
from tqdm import tqdm

In [4]:
class VoseAlias(object): # alis sampling
    """
    构建alias table,达到O(1)的采样效率
    Adding a few modifs to https://github.com/asmith26/Vose-Alias-Method
    """

    def __init__(self, dist):
        """
        初始化函数
        (VoseAlias, dict) -> NoneType
        """
        self.dist = dist
        self.alias_initialisation()

    def alias_initialisation(self):
        """
        Construct probability and alias tables for the distribution.
        """
        # Initialise variables
        n = len(self.dist)
        # 概率表
        self.table_prob = {}   # probability table
        # 替身表
        self.table_alias = {}  # alias table
        # 乘以n的概率表
        scaled_prob = {}       # scaled probabilities
        # 存储概率值小于1的
        small = []             # stack for probabilities smaller that 1
        # 存储概率值大于1的
        large = []             # stack for probabilities greater than or equal to 1

        # Construct and sort the scaled probabilities into their appropriate stacks
        # 将各个概率分成两组，一组的概率值大于1，另一组的概率值小于1
        print("1/2. Building and sorting scaled probabilities for alias table...")
        for o, p in tqdm(self.dist.items()):
            scaled_prob[o] = Decimal(p) * n

            if scaled_prob[o] < 1:
                small.append(o)
            else:
                large.append(o)

        print("2/2. Building alias table...")
        # Construct the probability and alias tables
        # 使用贪心算法，将概率值小于1的不断填满
        while small and large:
            s = small.pop()
            l = large.pop()

            self.table_prob[s] = scaled_prob[s]
            self.table_alias[s] = l
            # 更新概率值
            scaled_prob[l] = (scaled_prob[l] + scaled_prob[s]) - Decimal(1)

            if scaled_prob[l] < 1:
                small.append(l)
            else:
                large.append(l)

        # The remaining outcomes (of one stack) must have probability 1
        # 当两方不全有元素时，仅有一方有元素的也全为1
        while large:
            self.table_prob[large.pop()] = Decimal(1)

        while small:
            self.table_prob[small.pop()] = Decimal(1)
        self.listprobs = list(self.table_prob)

    def alias_generation(self):
        """
        Yields a random outcome from the distribution.
        """
        # Determine which column of table_prob to inspect
        col = random.choice(self.listprobs)
        # Determine which outcome to pick in that column
        # 取自己 
        if self.table_prob[col] >= random.uniform(0, 1):
            return col
        # 取替身，即取alias table存的节点
        else:
            return self.table_alias[col]

    def sample_n(self, size):
        """
        调用alias_generation一共n次，采样n个nodes
        Yields a sample of size n from the distribution, and print the results to stdout.
        """
        for i in range(size):
            yield self.alias_generation()

In [28]:
def makeDist(graphpath, power=0.75):
    # 读图函数
    # 初始化词典
    edgedistdict = collections.defaultdict(int)
   # print(edgedistdict)   defaultdict(<class 'int'>, {})
    nodedistdict = collections.defaultdict(int)

    weightsdict = collections.defaultdict(int)
    nodedegrees = collections.defaultdict(int)

    weightsum = 0
    negprobsum = 0
    # 统计图一共有多少条边
    nlines = 0
    
    with open(graphpath, "r") as graphfile:
        for l in graphfile:
            nlines += 1
    print(nlines)
    print("Reading edgelist file...")
    maxindex = 0
    with open(graphpath, "r") as graphfile:
        # tqdm能展示for循环进度百分比
        for l in tqdm(graphfile, total=nlines):
            # 点i,点j,weight
            line = [int(i) for i in l.replace("\n", "").split(" ")]
            node1, node2, weight = line[0], line[1], line[2]
          #  print(weight)1 1
            # 后面会做归一化，存的是归一化的边-权重和点-出度
            edgedistdict[tuple([node1, node2])] = weight
            nodedistdict[node1] += weight # 每个节点权重和
            print("node为",node1,nodedistdict[node1])
            
            # 不再做处理，存的是边-权重，点-出度
            weightsdict[tuple([node1, node2])] = weight
            nodedegrees[node1] += weight
            
            # weightsum存的是全图所有边的边权和，论文公式（2）中用到的1st相似度真实值
            weightsum += weight
            negprobsum += np.power(weight, power)
            
            # maxindex记录图中最大顶点index
            if node1 > maxindex:
                maxindex = node1
            elif node2 > maxindex:
                maxindex = node2
    for node, outdegree in nodedistdict.items():
        nodedistdict[node] = np.power(outdegree, power) / negprobsum

    for edge, weight in edgedistdict.items():
        edgedistdict[edge] = weight / weightsum

    return edgedistdict, nodedistdict, weightsdict, nodedegrees, maxindex

In [29]:
graph_path='../data/weighted.karate.edgelist'
edgedistdict, nodedistdict, weightsdict, nodedegrees, maxindex=makeDist(graph_path)

78
Reading edgelist file...


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 78/78 [00:00<00:00, 12972.59it/s]

node为 1 1
node为 1 2
node为 1 3
node为 1 4
node为 1 5
node为 1 6
node为 1 7
node为 1 8
node为 1 9
node为 1 10
node为 1 11
node为 1 12
node为 1 13
node为 1 14
node为 1 15
node为 1 16
node为 2 1
node为 2 2
node为 2 3
node为 2 4
node为 2 5
node为 2 6
node为 2 7
node为 2 8
node为 3 1
node为 3 2
node为 3 3
node为 3 4
node为 3 5
node为 3 6
node为 3 7
node为 3 8
node为 4 1
node为 4 2
node为 4 3
node为 5 1
node为 5 2
node为 6 1
node为 6 2
node为 6 3
node为 7 1
node为 9 1
node为 9 2
node为 9 3
node为 10 1
node为 14 1
node为 15 1
node为 15 2
node为 16 1
node为 16 2
node为 19 1
node为 19 2
node为 20 1
node为 21 1
node为 21 2
node为 23 1
node为 23 2
node为 24 1
node为 24 2
node为 24 3
node为 24 4
node为 24 5
node为 25 1
node为 25 2
node为 25 3
node为 26 1
node为 27 1
node为 27 2
node为 28 1
node为 29 1
node为 29 2
node为 30 1
node为 30 2
node为 31 1
node为 31 2
node为 32 1
node为 32 2
node为 33 1


In [17]:
edgedistdict

defaultdict(int,
            {(1, 32): 0.01282051282051282,
             (1, 22): 0.01282051282051282,
             (1, 20): 0.01282051282051282,
             (1, 18): 0.01282051282051282,
             (1, 14): 0.01282051282051282,
             (1, 13): 0.01282051282051282,
             (1, 12): 0.01282051282051282,
             (1, 11): 0.01282051282051282,
             (1, 9): 0.01282051282051282,
             (1, 8): 0.01282051282051282,
             (1, 7): 0.01282051282051282,
             (1, 6): 0.01282051282051282,
             (1, 5): 0.01282051282051282,
             (1, 4): 0.01282051282051282,
             (1, 3): 0.01282051282051282,
             (1, 2): 0.01282051282051282,
             (2, 31): 0.01282051282051282,
             (2, 22): 0.01282051282051282,
             (2, 20): 0.01282051282051282,
             (2, 18): 0.01282051282051282,
             (2, 14): 0.01282051282051282,
             (2, 8): 0.01282051282051282,
             (2, 4): 0.01282051282051282,
    

In [23]:
nodedistdict

defaultdict(int,
            {1: 0.10256410256410256,
             2: 0.060984980256549796,
             3: 0.060984980256549796,
             4: 0.029224449448138172,
             5: 0.021561446544967038,
             6: 0.029224449448138172,
             7: 0.01282051282051282,
             9: 0.029224449448138172,
             10: 0.01282051282051282,
             14: 0.01282051282051282,
             15: 0.021561446544967038,
             16: 0.021561446544967038,
             19: 0.021561446544967038,
             20: 0.01282051282051282,
             21: 0.021561446544967038,
             23: 0.021561446544967038,
             24: 0.04286796826771936,
             25: 0.029224449448138172,
             26: 0.01282051282051282,
             27: 0.021561446544967038,
             28: 0.01282051282051282,
             29: 0.021561446544967038,
             30: 0.021561446544967038,
             31: 0.021561446544967038,
             32: 0.021561446544967038,
             33: 0.01282

In [6]:
def negSampleBatch(sourcenode, targetnode, negsamplesize, weights,
                   nodedegrees, nodesaliassampler, t=10e-3):
    """
    For generating negative samples.
    """
    negsamples = 0
    while negsamples < negsamplesize:
        # nodesaliassampler是实现alias building的VoseAlias类，这里采样点
        samplednode = nodesaliassampler.sample_n(1)
        # 如果采样出source或target均跳过
        if (samplednode == sourcenode) or (samplednode == targetnode):
            continue
        # 输出负样本点，一共negsamplesize个点
        else:
            negsamples += 1
            yield samplednode

In [7]:
def makeData(samplededges, negsamplesize, weights, nodedegrees, nodesaliassampler):
    for e in samplededges:
        sourcenode, targetnode = e[0], e[1]
        negnodes = []
        # 采样出negsamplesize个负样本点
        for negsample in negSampleBatch(sourcenode, targetnode, negsamplesize,
                                        weights, nodedegrees, nodesaliassampler):
            for node in negsample:
                negnodes.append(node)
        # 格式是(node i, node j, negative nodes...)
        yield [e[0], e[1]] + negnodes